In [9]:
from pathlib import Path
from db import get_connection, close_connection
from base_api import get_api_instance
import pandas as pd

def insert_market_data(df: pd.DataFrame):
    connection = get_connection()
    try:
        with connection.cursor() as cursor:
            sql = """
            INSERT INTO market_data (
                `symbol`, `interval`, `datetime`, `open`,
                `high`, `low`, `close`, `volume`,
                `close_time`, `quote_asset_volume`, `number_of_trades`, `taker_buy_base_asset_volume`,
                `taker_buy_quote_asset_volume`, `ignore`
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                `open` = VALUES(`open`),
                `high` = VALUES(`high`),
                `low` = VALUES(`low`),
                `close` = VALUES(`close`),
                `volume` = VALUES(`volume`),
                `close_time` = VALUES(`close_time`),
                `quote_asset_volume` = VALUES(`quote_asset_volume`),
                `number_of_trades` = VALUES(`number_of_trades`),
                `taker_buy_base_asset_volume` = VALUES(`taker_buy_base_asset_volume`),
                `taker_buy_quote_asset_volume` = VALUES(`taker_buy_quote_asset_volume`),
                `ignore` = VALUES(`ignore`);
            """
            # Iterating over the DataFrame rows
            for index, row in df.iterrows():
                cursor.execute(sql, (
                    row['symbol'],
                    row['interval'],
                    row['datetime'], 
                    row['open'], 
                    row['high'], 
                    row['low'], 
                    row['close'], 
                    row['volume'], 
                    row['close_time'], 
                    row['quote_asset_volume'], 
                    row['number_of_trades'], 
                    row['taker_buy_base_asset_volume'], 
                    row['taker_buy_quote_asset_volume'], 
                    row['ignore']
                ))
        connection.commit()
    finally:
        close_connection(connection)


# markets = ["binance", "okx", "bingx", "bitget"]
# symbols = ["BTCUSDT", "ETHUSDT"]  # Example symbols
# intervals = ["1d", "30m"]   # Example intervals
# start_time='2023-12-01'
# end_time='2024-08-17'
symbols = ["BTCUSDT", "ETHUSDT", "XRPUSDT"]
market = "binance"
start_time='2017-12-01'
end_time='2024-09-06'
interval = "1d"
for symbol in symbols:
    api = get_api_instance(market)
    df = api.get_ohlc_data(symbol=symbol, interval=interval, start_time=start_time, end_time=end_time)
    df = df.assign(
        symbol=lambda x: symbol,
        interval=lambda x: interval
    )
    insert_market_data(df=df)
    print('done', symbol)
# data = api.load_data(symbol, interval)
# if data is None:
#     # api.save_data(df, symbol, interval, start_time, end_time)
#     # print(f"Saved data for {market} - {symbol} - {interval}")
# else:
#     print(f"Data already exists for {market} - {symbol} - {interval}")
# # load_data(market="binance", symbol="BTCUSDT", interval=interval, start_time=start_time, end_time=end_time)

done BTCUSDT
done ETHUSDT
done XRPUSDT


In [8]:
df

,datetime,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,symbol,interval
0,2017-12-01,9837.00000000,10898.00000000,9380.00000000,10782.99000000,6134.92363300,1512172799999,62260697.58291551,32375,3269.86858800,33193672.16592109,0,BTCUSDT,1d
1,2017-12-02,10775.04000000,11190.00000000,10620.00000000,10890.01000000,4765.43975700,1512259199999,52046689.84006951,29694,2390.19427500,26111051.97904352,0,BTCUSDT,1d
2,2017-12-03,10902.69000000,11825.00000000,10500.00000000,11165.41000000,5346.63652400,1512345599999,60350708.29332783,39335,2574.24618700,29090180.16290429,0,BTCUSDT,1d
3,2017-12-04,11165.41000000,11600.00000000,10802.00000000,11579.00000000,4663.42456200,1512431999999,52814985.63993067,32232,2516.85503100,28512751.47162289,0,BTCUSDT,1d
4,2017-12-05,11571.03000000,11853.00000000,11447.68000000,11699.99000000,5550.73205500,1512518399999,64721253.04527842,31317,3135.63056500,36570499.65542363,0,BTCUSDT,1d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,2024-09-01,58974.00000000,59076.59000000,57201.00000000,57301.86000000,20705.15741000,1725235199999,1202176847.39105850,2153180,9651.50531000,560452387.89854740,0,BTCUSDT,1d
2467,2024-09-02,57301.77000000,59425.69000000,57128.00000000,59132.13000000,22895.01461000,1725321599999,1333092369.44656860,1966119,11295.25452000,657496577.63048030,0,BTCUSDT,1d
2468,2024-09-03,59132.12000000,59809.65000000,57415.00000000,57487.73000000,22828.18447000,1725407999999,1335076815.85992480,2208758,10979.79204000,642252673.11321130,0,BTCUSDT,1d
2469,2024-09-04,57487.74000000,58519.00000000,55606.00000000,57970.90000000,35560.82146000,1725494399999,2027630870.17967540,3177549,16861.75483000,961923197.77825300,0,BTCUSDT,1d
